In [1]:
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization
import tensorflow.keras.preprocessing.image 

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7100406014087264509
]


In [6]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

[]


In [5]:
path = "E:\deep learning project\inaturalist_12K"

In [11]:
def get_data(path,augmentation=True):
    train_path=os.path.join(path,"train")
    test_path=os.path.join(path,"val")
    if augmentation==False:
        train_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.1)
    else:
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    train_data = train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    valid_data=train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    test_data=test_generator.flow_from_directory(
    directory=test_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    return train_data,valid_data,test_data

In [12]:
image_size=256
output_size=10

In [13]:
##from wandb.keras import WandbCallback

class CNN(object):
    def __init__(self,no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size=32,activation_func='relu',
               no_filters=32,image_size=256,drop_out=0.2,output_size=10,
               augmentation= True,batch_normalization=True):
        self.kenel_size=kernel_size
        self.learning_rate=learning_rate
        self.image_size=image_size
        self.output_size=output_size
        self.augmentation=augmentation
        self.batch_normalization=batch_normalization
        self.no_conv_layers=no_conv_layers
        self.dense_layer_size=dense_layer_size
        self.drop_out=drop_out
        self.no_filters=no_filters
        self.epochs=epochs
        self.padding=padding
        self.filter_no_metric= filter_no_metric
        self.initialize(no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size=32,activation_func='relu',
                    no_filters=32,image_size=256,drop_out=0.2,output_size=10,
                    augmentation= True,batch_normalization=True)
    def initialize(self,no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size=32,activation_func='relu',
                 no_filters=32,image_size=256,drop_out=0.2,output_size=10,
                    augmentation= True,batch_normalization=True
                 ):
        self.model=Sequential()
        for i in range(0,no_conv_layers):
            if i==0:
                self.model.add(Conv2D(no_filters,kernel_size, input_shape=(image_size, image_size, 3),kernel_initializer = "he_uniform",padding = padding,
                           data_format="channels_last"))
            else:
                if filter_no_metric=="1":
                    self.model.add(Conv2D(no_filters,kernel_size,kernel_initializer = "he_uniform",padding =padding))
                elif filter_no_metric=="2":
                    self.model.add(Conv2D(no_filters*(2**i),kernel_size,kernel_initializer = "he_uniform",
                                          padding =padding))
                elif filter_no_metric=="1/2":
                    self.model.add(Conv2D(no_filters*(1/(2**i)),kernel_size,kernel_initializer = "he_uniform",
                                          padding =padding))

            self.model.add(Activation(activation_func))
            if batch_normalization==True:
                self.model.add(BatchNormalization())
            self.model.add(MaxPooling2D(pool_size=(2,2)))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Flatten())
        self.model.add(Dense(dense_layer_size))
        self.model.add(Activation(activation_func))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))




In [14]:
import numpy as np
from tensorflow.keras.optimizers import Adam
np.random.seed(2)
classes=["Amphibia","Animalia","Arachnida","Aves","Fungi","Insecta","Mammalia","Mollusca","Plantae","Reptilia"]
train_data,valid_data,test_data= get_data(path,True)
best_model=CNN(5,(3,3),0.001,1,'same','1',64,'elu',
              128,image_size,0.4,output_size,True,True)
best_model.model.compile(
optimizer=Adam(0.001),  # Optimizer
loss="categorical_crossentropy", metrics="categorical_accuracy")
  #train_data,valid_data,test_data=get_data(path,augmentation=True)
best_model.model.fit(train_data,epochs=1,batch_size=32,validation_data=valid_data)

Found 9999 images belonging to 10 classes.
Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


UnknownError:  Failed to get convolution plans. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_2/activation_14/Relu (defined at \AppData\Local\Temp\ipykernel_23708\2316745762.py:12) ]] [Op:__inference_train_function_6082]

Function call stack:
train_function
